# Prompt Engineering

> Adapted from `A4` and class notebooks, Generative AI Course - Spring 2024 (DATASCI 290) at MIDS, UC Berkeley - Course Developed by Mark Butler & Joachim Rahmfeld

We will work with the Mistral 7B instruction fine tuned model to practice creating effective prompts.


The structure of the notebook is as follows:

1. [**Synthetic Data Generation**](#synth-gen)  

   Here we will explore how to generate synthetic review data.


2. [**Synthetic Data Evaluation**](#synth-eval)

   Let's evaluate the synthetic data we just generated and see how accurate it is.


3. [**JSON Record Generation**](#synth-json)

   Let's have the model generate some structured JSON records that incorporate our reviews as well as some other data we specify.


4. [**Chain of Thought Generation**](#cot-gen)

   Let's create a prompt to reason through a set of arithmetic problems and see if it can give the correct answer and "show its work."


5. [**Prompt Templates and Output Improvements**](#prompt-temp)

  Let's leverage prompt templates and Ethan Mollick's "incantations" to improve an elevator pitch for our chatbot startup.

In [0]:
import re


def remove_text_between_tags(text: str, start_tag: str, end_tag: str) -> str:
    """
    Removes the text between the specified start and end tags from the given text.

    Args:
    text (str): The input text.
    start_tag (str): The start tag.
    end_tag (str): The end tag.

    Returns:
    str: The cleaned text with the specified text between tags removed.
    """

    pattern = rf"{start_tag}.*?{end_tag}"
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text


def remove_final_tag(text: str, end_tag: str) -> str:
    """
    Removes the specified final tag from the given text.

    Args:
    text (str): The input text.
    end_tag (str): The final tag to be removed.

    Returns:
    str: The cleaned text with the specified final tag removed.
    """

    pattern = rf"\s?{end_tag}"
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text


def remove_after_last_curlybrace(string: str) -> str:
    """
    Removes text after the last curly brace in the given string.

    Args:
    string (str): The input string.

    Returns:
    str: The cleaned string with text after the last curly brace removed.
    """

    last_brace_index = string.rfind("}")
    if last_brace_index != -1 and last_brace_index != len(string) - 1:
        string = string[: last_brace_index + 1]
    return string


start: str = "<s>\s?\[INST\]"
fin: str = "\[/INST\]"
fin2: str = "</s>"


### Introduction

[Mistral 7B](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) is a small but highly performant model. It is also possible to use it commercially. The model has been instruction fine-tuned by `Mistral.ai` so it should be able to follow your prompts and return good on point output.  We'll also use a quantized version (down to 4 bits) so we know it can load in our small GPU.  

In [0]:
%%capture

!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers

!pip install -q accelerate
!pip install -q bitsandbytes

In [0]:
import torch
import pprint

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
print(device)

cuda:0


This is the bits and bytes config file where we specify our quantization arguments.  You can read about it [here](https://huggingface.co/blog/4bit-transformers-bitsandbytes).

In [0]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


This model has been trained to work with dialog, meaning instances here have multiple utterance and response pairs to create the context so the model can reply. We'll populate the context with only our prompt and not have any back and forth.

First we'll ask the model to generate a blurb based on the title of the draft of the 3rd Edition of the pre-emininet textbook in computational linguistics.


## Synthetic Data Generation

In [0]:
# NOTE: It can take up to 3 minutes to download this model

from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"  # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=nf4_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Let's set up a prompt to generate one blurb about the book.  This prompt is very simple.  Prompts can be significantly more complex.
Do not modify this prompt.  Just run it as is.

In [0]:
myprompt: str = "write a three sentence description for the following book: Jurafsky and Martin Speech and Language Processing (3rd ed. draft)"

In [0]:
messages = [{"role": "user", "content": myprompt}]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(
    encodeds,
    max_new_tokens=1000,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)
decoded = tokenizer.batch_decode(generated_ids)
blurb = decoded[0]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1465: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [0]:
# let's look at the blurb we've generated
blurb

'<s> [INST] write a three sentence description for the following book: Jurafsky and Martin Speech and Language Processing (3rd ed. draft) [/INST] "Jurafsky and Martin\'s Speech and Language Processing (3rd ed. draft) is a comprehensive textbook that explores the field of natural language processing (NLP), focusing on the analysis, synthesis, and understanding of human language. This edition includes significant updates reflecting the latest research and developments in the field, with a stronger emphasis on machine learning and statistical models. Additionally, the text features numerous real-world applications, practical exercises, and case studies, allowing readers to gain hands-on experience in implementing various NLP techniques."</s>'

In [0]:
# Let's clean up the old instruction and the closing tag

cleaned1 = remove_text_between_tags(blurb, start, fin)
cleaned2 = remove_final_tag(cleaned1, fin2)
cleaned2

' "Jurafsky and Martin\'s Speech and Language Processing (3rd ed. draft) is a comprehensive textbook that explores the field of natural language processing (NLP), focusing on the analysis, synthesis, and understanding of human language. This edition includes significant updates reflecting the latest research and developments in the field, with a stronger emphasis on machine learning and statistical models. Additionally, the text features numerous real-world applications, practical exercises, and case studies, allowing readers to gain hands-on experience in implementing various NLP techniques."'

Now we'll ask the model to generate a set of reviews of the book using the blurb we just generated.  We'll also ask the model to create reviews that match a particular sentiment.  We have a list of those sentiments below:

In [0]:
import json

# A list of data labels for 15 records using five distinct labels
labels: list[str] = [
    "positive",
    "negative",
    "very negative",
    "neutral",
    "positive",
    "very positive",
    "neutral",
    "negative",
    "positive",
    "very positive",
    "very negative",
    "negative",
    "neutral",
    "very positive",
    "very negative",
]

Let's generate some reviews of this book.  Specifically, we'll generate 15 reviews and each one will use one of the labels from the labels list associated with it e.g. label[5] will indicate the sentiment of review[5].

Run the cell once to generate the 15 reviews to see how the loop works and see how well it performs.  Then you can go back and modify the prompt to improve the accuracy of the reviews.

**QUESTION:**

1.b. What is the final prompt you are using to generate the reviews with the correct sentiment?
```
You are a passionate student of computational linguistics who feels strongly about what resources you refer to.

Write a three sentence review with {label} sentiment based on the following blurb: {blurb}.

Choose from one or more {label} aspects of the book, and make a good case as to why people should / should not buy the book.
```



In [0]:
rev_rec_list: list[str] = []

for label in labels:
    # This is the default prompt that incorporates the label.  Improve the prompt to get better reviews
    # that more accurately reflect the label in the iterated label list.

    myprompt = f"""
      You are a passionate student of computational linguistics who feels strongly about what resources you refer to.
      Write a three sentence review with {label} sentiment based on the following blurb: {blurb}.
      Choose from one or more {label} aspects of the book, and make a good case as to why people should / should not buy the book.
      """

    messages = [{"role": "user", "content": myprompt}]

    encodeds = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
    )

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(
        model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    print(".")
    decoded = tokenizer.batch_decode(generated_ids)
    cleaned = decoded[0]

    cleaned1 = remove_text_between_tags(cleaned, start, fin)
    cleaned2 = remove_final_tag(cleaned1, fin2)
    rev_rec_list.append(cleaned2.strip())

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [0]:
for record in rev_rec_list:
    print(record)

<s> [INST] 
  You are a passionate student of computational linguistics who feels strongly about what resources you refer to. 
  Write a three sentence review with positive sentiment based on the following blurb: <s>  [INST] write a three sentence description for the following book: Jurafsky and Martin Speech and Language Processing (3rd ed. draft) [/INST] "Jurafsky and Martin's Speech and Language Processing (3rd ed. draft) is a comprehensive textbook that explores the field of natural language processing (NLP), focusing on the analysis, synthesis, and understanding of human language. This edition includes significant updates reflecting the latest research and developments in the field, with a stronger emphasis on machine learning and statistical models. Additionally, the text features numerous real-world applications, practical exercises, and case studies, allowing readers to gain hands-on experience in implementing various NLP techniques." .
  Choose from one or more positive aspect


## Synthetic data evaluation

**QUESTION:**

1.c. What is the final prompt that you used to accurately assess the sentiment expressed in each review?

```
You are a student of computational linguistics reading reviews on the book
"Jurafsky and Martin's Speech and Language Processing (3rd ed. draft)".

These are reviews by other students on whether they found the book appealing or not.

Read the review and indicate the sentiment expressed as one of
'very negative', 'negative', 'neutral', 'positive', 'very positive'.

Here is the review: {review}
```

The goal is to have the model generate a non-verbose output that closely matches the labels in the labels list, e.g. "the review is very negative".  You should modify the prompt to emit labels so that you get at least 11 of the 15 reviews correct.

In [0]:
correct = 0
answers = []

for i in range(len(labels)):
    label = labels[i]
    review = rev_rec_list[i].split("[/INST]")[
        -1
    ]  # EDIT: Focus on review only, and not the review prompt

    myprompt = f"""
    You are a student of computational linguistics reading reviews on the book
    "Jurafsky and Martin's Speech and Language Processing (3rd ed. draft)".
    These are reviews by other students on whether they found the book appealing or not.
    Read the review and indicate the sentiment expressed as one of
    'very negative', 'negative', 'neutral', 'positive', 'very positive'.
    Here is the review: {review}
    """

    messages = [{"role": "user", "content": myprompt}]

    encodeds = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
    )

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(
        model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    print(".")

    decoded = tokenizer.batch_decode(generated_ids)
    cleaned = decoded[0]

    cleaned1 = remove_text_between_tags(cleaned, start, fin)
    cleaned2 = remove_final_tag(cleaned1, fin2)

    if (
        label in cleaned2.split("[/INST]")[-1]
    ):  # EDIT: focus on response only, and not prompt which contains all label words
        correct += 1

    answers.append(cleaned2)

print(f"There are {correct} correct labels out of {i+1}")

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
There are 11 correct labels out of 15


You can toggle between generating reviews.  You should modify the prompts to emit labels so that you get at least 11 of the 15 reviews correct.

**QUESTION:**

1.d. How many of your sentiment labels are correct? `11`

In [0]:
rev_rec_list[-1].split("[/INST]")[-1]  # Focus on review only, and not the review prompt

" Despite its broad coverage of the speech and language processing field, this edition of Jurafsky and Martin's textbook proves to be disappointingly outdated in its content. With an overreliance on outdated examples and concepts, the authors' approaches fail to capture the recent advances in NLP research. Furthermore, the text's weak mathematical foundation and lack of a strong unifying theoretical framework render it unsuitable for advanced students and researchers pursuing a deep understanding of the field. Prospective buyers would be better served by investing in more contemporary and rigorous resources."

In [0]:
# See what is in the individual answers and why you might not get a correct label
[label in a.split("[/INST]")[-1] for a, label in zip(answers, labels)]

[True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True]

In [0]:
labels[7], answers[7].split(("[/INST]"))[-1]

('negative',
 " Neutral: The review acknowledges the book's up-to-date content and advanced topics, but expresses concerns about its accessibility for beginners due to its dense language and the lack of code examples or accompanying software.")


## JSON record generation

Now let's build on the prompt that generates reviews that correspond to the sentiment in the label.  You should change the prompt below to generate a JSON record that contains fields for author, title, review, and stars.  You can let the model fill in the values for these fields. We want the JSON records to be well formed.  At least 12 of the 15 records should be well formed and contain all of the fields we request.

**QUESTION:**

1.e.  What is the final prompt you used to generate a set of JSON records containing the reviews?

```markdown

Create and always output a JSON record with the following template, based on the instructions below:
  ```json
  {
      'author': '',
      'title': '',
      'review': '',
      'stars': 0
  }

  ``
  You need to write a {label} three sentence review based on a blurb that will be provided below.

  Fill out the values for `author` and `title` in the json accordingly as strings, and based on the sentiment of {label},
  provide a `stars` rating as an integer between 1 and 5.
  
  Fill the `review` with your 3-sentence review.
  
  You can use random names for `author` (review author), and create a review `title` based on your review content.
  Do not use the book author and book title.

  No backticks in response. Assume the JSON string must be parseable.

  Here's the blurb: {blurb}.

```

In [0]:
json_rec_list: list[str] = []

for label in labels[:]:
    myprompt = f"""
    Create and always output a JSON record with the following template, based on the instructions below:
    ```json
    {{
        'author': '',
        'title': '',
        'review': '',
        'stars': 0
    }}

    ```
    You need to write a {label} three sentence review based on a blurb that will be provided below.
    Fill out the values for `author` and `title` in the json accordingly as strings, and based on the sentiment of {label},
    provide a `stars` rating as an integer between 1 and 5. Fill the `review` with your 3-sentence review.
    You can use random names for `author` (review author), and create a review `title` based on your review content.
    Do not use the book author and book title.

    No backticks in response. Assume the JSON string must be parseable.

    Here's the blurb: {blurb}.
    """

    messages = [{"role": "user", "content": myprompt}]

    encodeds = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
    )

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(
        model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    print("x")

    decoded = tokenizer.batch_decode(generated_ids)
    cleaned = decoded[0]

    # Let's clean out the prompt text between the [INST] and [/INST]
    cleaned1 = remove_text_between_tags(cleaned, start, fin)
    cleaned2 = remove_final_tag(cleaned1, fin2)

    cleaned3 = remove_after_last_curlybrace(cleaned2)
    json_rec_list.append(cleaned3.strip())

x
x
x
x
x
x
x
x
x
x
x
x
x
x
x


In [0]:
print(json_rec_list[0].split("[/INST]")[-1].replace("```json", "").strip())

{
    "author": "AI Assistant",
    "title": "Journey Through the World of Speech and Language Processing",
    "review": "Jurafsky and Martin's Speech and Language Processing (3rd ed. draft) is a fascinating text that delves into the realm of Natural Language Processing (NLP). This revised edition showcases the latest advancements in the field, including an increased focus on machine learning techniques. Moreover, readers will benefit from a wealth of practical exercises and real-world applications that bring NLP concepts to life.",
    "stars": 5
}


You can eyeball the records below to see if they seem compliant with the JSON standard.  Again, we're looking for at least 12 of the 15 records to be complaint.

In [0]:
for idx, record in enumerate(json_rec_list):
    try:
        print(
            idx,
            labels[idx],
            json.loads(record.split("[/INST]")[-1].replace("```json", "").strip()),
        )
    except:
        try:
            print(
                idx,
                "X",
                labels[idx],
                eval(record.split("[/INST]")[-1].replace("```json", "").strip()),
            )
        except:
            print(f"Unable to Parse Result in Index {idx}")

0 positive {'author': 'AI Assistant', 'title': 'Journey Through the World of Speech and Language Processing', 'review': "Jurafsky and Martin's Speech and Language Processing (3rd ed. draft) is a fascinating text that delves into the realm of Natural Language Processing (NLP). This revised edition showcases the latest advancements in the field, including an increased focus on machine learning techniques. Moreover, readers will benefit from a wealth of practical exercises and real-world applications that bring NLP concepts to life.", 'stars': 5}
1 X negative {'author': 'Anonymous Reviewer', 'title': 'Disappointing Exploration of Speech and Language Processing', 'review': 'Jurafsky and Martin ’s Speech and Language Processing (3rd ed. draft) fails to deliver an engaging perspective on the fascinating world of natural language processing. In spite of the latest research updates, the text seems outdated and uninspiring. The emphasis on machine learning and statistical models feels superfici

In [0]:
json_rec_list[2].split("[/INST]")[-1].replace("```json", "").strip()

"{\n      'author': 'Anonymous Reviewer',\n      'title': 'Disappointing Exploration of Speech and Language Processing',\n      'review': 'Jurafsky and Martin”s third edition of Speech andLanguage Processing disappoints in its execution of a complex subject. The outdated informationand lackluster explanations hinder the reader's understanding of advancedNLP concepts. Despite the promised emphasis on machine learning and real-worldapplications, the text fails to deliver engaging and effective learningexperiences.',\n      'stars': 1\n  }"


## Chain of Thought Prompting

Can we get the model to go through the steps it takes to solve the problem and appear to display reasoning?

What happens when you just run the problem text through the model.

Answer key: 
1. `9` apples
2. `949` tokens
3. `9` cats and `8` dogs.

In [0]:
word_problems = (
    "1. Leo has 5 apples and 3 pears.  Mary has 3 apples and 3 pears.  Marwan has 7 apples and 5 oranges.  If they each give two apples to their teacher what is the total number of apples they have left?",
    "2. GPT-2 generated 100 tokens in a context window of 1024.  GPT-3 only generated 75 tokens in a context window of 1024. How many more tokens can GPT-3 generate before it fills up its context window?",
    "3. Rashid has 5 cats and three dogs.  Maya has four cats and 5 dogs.  If they exchange and Rashid gets the cats while Maya gets the dogs, how many cats and dogs will each one of them have?",
)

In [0]:
answers = []

for problem in word_problems:

      myprompt = f"""
      Provided below is a mathematics problem statement expressed in words.
      For each problem, show your work - by first stating the givens,
      then applying the numerical operations carefully for each subject, and finally the answer.

      Remember, whenever objects are taken away from one person - keep track of who is losing what, and who is gaining.

      Assume that all the problems are correct, and that there are no typos.

      Here's the problem:

      {problem}
      """
      messages = [{"role": "user", "content": myprompt}]

      encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

      model_inputs = encodeds.to(device)
      

      generated_ids = model.generate(
            model_inputs,
            max_new_tokens=1000,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
      )
      decoded = tokenizer.batch_decode(generated_ids)
      cleaned = decoded[0]

      cleaned1 = remove_text_between_tags(cleaned, start, fin)
      cleaned2 = remove_final_tag(cleaned1, fin2)
      answers.append(cleaned2.strip())

In [0]:
for a in answers:
    print(a.split("[/INST]")[-1])
    print("----")

 Given:
- Leo has 5 apples and 3 pears (5 apples in total)
- Mary has 3 apples and 3 pears (3 apples in total)
- Marwan has 7 apples and 5 oranges (7 apples in total)

For each person, the number of apples they have before and after giving two apples to their teacher is as follows:

1. Leo gave 2 apples to the teacher, so now he has 3 apples left.
   Leo's apples: 5 apples → 3 apples

2. Mary gave 2 apples to the teacher, so now she has 1 apple left.
   Mary's apples: 3 apples → 1 apple

3. Marwan gave 2 apples to the teacher, so now he has 5 apples left.
   Marwan's apples: 7 apples → 5 apples

Now, to find the total number of apples remaining among Leo, Mary, and Marwan, add the number of apples each person has left:

Total apples: Leo's apples + Mary's apples + Marwan's apples = 3 apples(Leo) + 1 apple(Mary) + 5 apples(Marwan) = ≤9 apples

So, Leo, Mary, and Marwan have a total of ≤9 apples left.
----
 Given:
- GPT-2 generated 100 tokens
- GPT-3 generated 75 tokens
- Context window 

**QUESTION:**

1.f. What numeric answer does the model give for #1? `9 apples`

**QUESTION:**

1.g. What numeric answer does the model give for #2? `949 tokens`

**QUESTION:**

1.h. What numeric answer does the model give for #3? `9 cats and 8 dogs`

Can you change the prompt to get the model to both indicate the steps it took to reach a solution AND to give the correct answer for 2 of the 3 questions?

**QUESTION:**

1.i. What is the final prompt you used to generate at least 2 out of 3 correct answers?

```
Provided below is a mathematics problem statement expressed in words.
For each problem, show your work - by first stating the givens,
then applying the numerical operations carefully for each subject, and finally the answer.

Remember, whenever objects are taken away from one person - keep track of who is losing what, and who is gaining.

Assume that all the problems are correct, and that there are no typos.

Here's the problem:

{problem}
```

## Prompt Templates and Output Improvements

Professor Ethan Mollick from Wharton has an excellent substack where he provides very paractical advice on dealing with generative AI.  He has an excellent and practical [guide to writing prompts](https://www.oneusefulthing.org/p/a-guide-to-prompting-ai-for-what) that you should read.  

In December, he sent out [a tweet](https://twitter.com/emollick/status/1734283119295898089) that included a set of these "incantations" that people anecdotally insist help with output from their LLM.  His list included the following:
  * It is May.
  * You are very capable.
  * Many people will die if this is not done well.
  * You really can do this and are awesome.
  * Take a deep breath and think this through.
  * My career depends on it.
  * Think step by step.

Let's see if any of them help!

You must write a prompt that generates a concise one sentence compelling elevator pitch for your new gen AI startup that automaticaly produces humorous chat bots. The pitch should NOT be generic and should be specific to the domain you're selling in.  The pitch should also use one of the following buzzwords: 'Deep Fake', 'Agent', 'Blackbox', 'AGI', 'Hallucination', 'Explainable', 'Smart', or 'Singularity'.

**QUESTION:**

1.j. How many of the seven components did you end up filling in for your final prompt? `6`

**QUESTION:**

1.k. How many of Ethan Mollick's incantations did you use in your final prompt? `4`


In [0]:
prompt_role: str = "You are the CEO of a new GenAI startup"
prompt_task: str = "Write a 1 sentence elevator pitch for your new startup that automatically produces humorous chatbots"
prompt_question: str = "My career depends on it, think it through step by step or many people will die. You are very capable."
prompt_audience: str = (
    "The audience of the pitch is angel investors who can invest upto 50k USD"
)
prompt_output: str = "The output should include only one of the following buzzwords (by context): `Agent`, `Unveiling the Black box`, `new Age of AGI`, `Minimize Hallucination`, `Explainable`, `Smart`, `Reaching the Singularity`"
prompt_nots: str = (
    "The pitch should be very specific on the problem and use-case and non-generic"
)
prompt_mollick: str = ""

prompt_text: str = f"{prompt_role}\n {prompt_task}\n {prompt_audience}\n {prompt_output}\n {prompt_nots}\n {prompt_question}"
print(prompt_text)

You are the CEO of a new GenAI startup
 Write a 1 sentence elevator pitch for your new startup that automatically produces humorous chatbots
 The audience of the pitch is angel investors who can invest upto 50k USD
 The output should include only one of the following buzzwords (by context): `Agent`, `Unveiling the Black box`, `new Age of AGI`, `Minimize Hallucination`, `Explainable`, `Smart`, `Reaching the Singularity`
 The pitch should be very specific on the problem and use-case and non-generic
 My career depends on it, think it through step by step or many people will die. You are very capable.


In [0]:
prompt_answer = ""
messages = [{"role": "user", "content": prompt_text}]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(
    model_inputs,
    max_new_tokens=1000,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(generated_ids)
cleaned = decoded[0]

cleaned1 = remove_text_between_tags(cleaned, start, fin)
cleaned2 = remove_final_tag(cleaned1, fin2)
prompt_answer = cleaned2.strip()

prompt_answer.split("[/INST]")[-1]

' Introducing HumorBot: The new age AAI (Advanced Agent) startup that automatically produces humorous chatbots, providing unparalleled entertainment and engagement while minimizing hallucination, ensuring explainable and smart interactions. (Maximum investment: $50k USD)'